In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

import tensorflow as tf
from keras import backend as K

from tensorflow.keras.preprocessing.text import text_to_word_sequence, Tokenizer
from tensorflow.keras.layers import Embedding, Input, Dense, LSTM, GRU, Bidirectional, TimeDistributed
from tensorflow.keras.models import Sequential, Model
import tensorflow.keras.initializers as initializers
import tensorflow.keras.regularizers as regularizers
import tensorflow.keras.constraints as constraints

In [3]:
MAX_SENTENCES = 50
MAX_WORDS_PER_SENTENCE = 50

In [5]:
def read_dataset(data_path):
    with open(data_path) as f:
        d_lines = f.read().splitlines()
        
    data = []
    labels = []
    for line in d_lines:
        features = line.split('<fff>')
        label, doc_id, sentences = int(features[0]), int(features[1]), features[2:]
        
        labels.append(label)
        
        doc_tokens = [] # contain tokens for every sentence in the doc
        for sent in sentences:
            sent_tokens = [int(token) for token in sent.split()]
            doc_tokens.append(sent_tokens)
            
        data.append(doc_tokens)
        
    return np.array(data), np.array(labels)

In [8]:
X_train, y_train = read_dataset('../input/attention-v2/20news-train-encoded.txt')
X_test, y_test = read_dataset('../input/attention-v2/20news-test-encoded.txt')

y_train = pd.get_dummies(pd.Series(y_train)).values
y_test = pd.get_dummies(pd.Series(y_test)).values

In [9]:
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

In [10]:
class AttentionWithContext(tf.keras.layers.Layer):
    
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [16]:
with open('../input/attention-set/vocab-raw.txt', 'rb') as f:
    vocab_size = len(f.read().splitlines())

In [20]:
embedding_layer = Embedding(
    input_dim=vocab_size+2, output_dim=300, input_length=MAX_WORDS_PER_SENTENCE
)

word_input = Input(shape=(MAX_WORDS_PER_SENTENCE, ), dtype='int32')
word_sequence = embedding_layer(word_input)
word_lstm = Bidirectional(LSTM(units=100, return_sequences=True))(word_sequence)
word_dense = TimeDistributed(Dense(200))(word_lstm)
word_att = AttentionWithContext()(word_dense)
wordEncoder = Model(word_input, word_att)

sent_input = Input(
    shape=(MAX_SENTENCES, MAX_WORDS_PER_SENTENCE), dtype='int32')
sent_encoder = TimeDistributed(wordEncoder)(sent_input)
sent_lstm = Bidirectional(LSTM(100, return_sequences=True))(sent_encoder)
sent_dense = TimeDistributed(Dense(200))(sent_lstm)
sent_att = AttentionWithContext()(sent_dense)
preds = Dense(20, activation='softmax')(sent_att)
model = Model(sent_input, preds)

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer='adam',
              metrics=['acc'])

In [21]:
model.fit(X_train, y_train, validation_split=0.1,
          epochs=10, batch_size=100)

Epoch 1/10
102/102 [==============================] - 30s 240ms/step - loss: 2.6236 - acc: 0.1170 - val_loss: 6.9210 - val_acc: 0.0654
Epoch 2/10
102/102 [==============================] - 23s 225ms/step - loss: 1.5084 - acc: 0.4394 - val_loss: 7.8405 - val_acc: 0.1890
Epoch 3/10
102/102 [==============================] - 23s 225ms/step - loss: 0.6280 - acc: 0.7816 - val_loss: 7.8750 - val_acc: 0.2014
Epoch 4/10
102/102 [==============================] - 23s 225ms/step - loss: 0.2018 - acc: 0.9392 - val_loss: 8.1863 - val_acc: 0.2173
Epoch 5/10
102/102 [==============================] - 23s 226ms/step - loss: 0.0857 - acc: 0.9748 - val_loss: 8.9849 - val_acc: 0.2226
Epoch 6/10
102/102 [==============================] - 23s 226ms/step - loss: 0.0517 - acc: 0.9856 - val_loss: 10.1406 - val_acc: 0.1970
Epoch 7/10
102/102 [==============================] - 23s 226ms/step - loss: 0.0259 - acc: 0.9927 - val_loss: 10.1168 - val_acc: 0.2111
Epoch 8/10
102/102 [==============================] -

In [22]:
model.evaluate(X_test, y_test)

236/236 [==============================] - 6s 27ms/step - loss: 2.1654 - acc: 0.7195


[2.1654274463653564, 0.7194636464118958]